In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [ ]:
# Step 1: Load your data from CSV files
# Replace 'DiabeticRetinopathy_train.csv', 'DiabeticRetinopathy_validation.csv', 'DiabeticRetinopathy_test.csv' with actual file paths
train_data = pd.read_csv('DiabeticRetinopathy_train.csv')
validation_data = pd.read_csv('DiabeticRetinopathy_validation.csv')
test_data = pd.read_csv('DiabeticRetinopathy_test.csv')

In [ ]:
# Step 2: Prepare input feature vectors (X) and target/class vectors (y)
# Assume the last column is the class label, and the rest are features
X_train = train_data.iloc[:, :-1].values
y_train = train_data.iloc[:, -1].values
X_val = validation_data.iloc[:, :-1].values
y_val = validation_data.iloc[:, -1].values
X_test = test_data.iloc[:, :-1].values
y_test = test_data.iloc[:, -1].values

In [ ]:
# Step 3: Standardize the feature data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_val, y_val = torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32).view(-1, 1)
X_test, y_test = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

## **Neural Network Implementation**

In [ ]:
# Step 4: Define the Neural Network model
# Architecture example: Input Layer -> Hidden Layer (64) -> Output Layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_layers):
        super(NeuralNet, self).__init__()
        layers = []
        prev_size = input_size
        for layer_size in hidden_layers:
            layers.append(nn.Linear(prev_size, layer_size))
            layers.append(nn.Sigmoid())
            prev_size = layer_size
        layers.append(nn.Linear(prev_size, 1))
        layers.append(nn.Sigmoid())
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [ ]:
# Step 5: Training function
def train_model(model, criterion, optimizer, epochs=100):
    train_losses = []
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            
        train_losses.append(epoch_loss / len(train_loader.dataset))
        
        if (epoch+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {train_losses[-1]:.4f}')
    
    return train_losses

# Step 6: Evaluation function
def evaluate_model(model, X, y):
    model.eval()
    with torch.no_grad():
        outputs = model(X)
        predicted = (outputs >= 0.5).float()
        y_np = y.numpy().flatten()
        pred_np = predicted.numpy().flatten()
        
        return {
            'confusion_matrix': confusion_matrix(y_np, pred_np),
            'accuracy': accuracy_score(y_np, pred_np),
            'precision': precision_score(y_np, pred_np),
            'recall': recall_score(y_np, pred_np),
            'f1': f1_score(y_np, pred_np)
        }

In [ ]:
# Step 7: Define architectures to test
architectures = [
    # Single hidden layer architectures
    {'name': '10-neuron', 'layers': [10]},
    {'name': '100-neuron', 'layers': [100]},
    {'name': '64-neuron', 'layers': [64]},
    {'name': '512-neuron', 'layers': [512]},
    
    # Two hidden layer architectures
    {'name': '64->16', 'layers': [64, 16]},
    {'name': '128->32', 'layers': [128, 32]},
    {'name': '256->64', 'layers': [256, 64]},
    {'name': '512->128', 'layers': [512, 128]}
]

# Step 8: Train and evaluate models
results = []
best_model = None
best_f1 = 0

for arch in architectures:
    print(f"\n=== Training {arch['name']} Architecture ===")
    
    # Initialize model
    model = NeuralNet(input_size=19, hidden_layers=arch['layers'])
    criterion = nn.MSELoss(reduction='sum')  # SSE Loss
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    
    # Train model
    train_losses = train_model(model, criterion, optimizer, epochs=100)
    
    # Evaluate
    val_metrics = evaluate_model(model, X_val, y_val)
    test_metrics = evaluate_model(model, X_test, y_test)
    
    # Store results
    results.append({
        'architecture': arch['name'],
        'train_loss': train_losses[-1],
        'val_accuracy': val_metrics['accuracy'],
        'val_f1': val_metrics['f1'],
        'test_accuracy': test_metrics['accuracy'],
        'test_f1': test_metrics['f1']
    })
    
    # Track best model
    if val_metrics['f1'] > best_f1:
        best_f1 = val_metrics['f1']
        best_model = model
        
    # Print metrics
    print(f"\n{arch['name']} Validation Metrics:")
    print(f"Accuracy: {val_metrics['accuracy']:.4f}, F1: {val_metrics['f1']:.4f}")
    print(f"Confusion Matrix:\n{val_metrics['confusion_matrix']}")
    
    print(f"\n{arch['name']} Test Metrics:")
    print(f"Accuracy: {test_metrics['accuracy']:.4f}, F1: {test_metrics['f1']:.4f}")
    print(f"Confusion Matrix:\n{test_metrics['confusion_matrix']}")


In [ ]:
# Step 9: Visualize results
results_df = pd.DataFrame(results)

# Plot performance comparison
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.bar(results_df['architecture'], results_df['val_f1'])
plt.title('Validation F1-Score by Architecture')
plt.xticks(rotation=45, ha='right')

plt.subplot(1, 2, 2)
plt.bar(results_df['architecture'], results_df['test_f1'])
plt.title('Test F1-Score by Architecture')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Display best model results
best_result = results_df[results_df['val_f1'] == best_f1].iloc[0]
print(f"\nBest Architecture: {best_result['architecture']}")
print(f"Validation F1: {best_result['val_f1']:.4f}")
print(f"Test F1: {best_result['test_f1']:.4f}")



## Step 5: Initialize the model with different architectures
### Example: model = NeuralNet(input_size=19, hidden_layers=[64, 16])

## Step 6: Define Loss Function and Optimizer
### criterion = nn.MSELoss()
### optimizer = optim.SGD(model.parameters(), lr=0.01)

## Step 7: Train the model using training data.
### Perform forward pass, compute loss, backpropagate, and update weights.

## Step 8: Evaluate the model on validation and test data.
### Compute confusion matrix, accuracy, precision, recall, and F1-score.

## Step 9: Experiment with different architectures and analyze the results.

## Step 10: Visualize training loss and performance metrics for comparison.